In [ ]:
import os

from DataLoader import (
    config,
    loader
)

import pandas as pd
import numpy as np
from scipy.stats import norm
from scipy.optimize import curve_fit

import matplotlib.pyplot as plt
import seaborn as sns

from copy import deepcopy
 
from sklearn.preprocessing import (
    MinMaxScaler,
    PolynomialFeatures
)
from sklearn.metrics import mean_squared_error

In [ ]:
# Я понял как нужно считывать данные, какой должна быть конечная структуру
data_path = '../data/raw/'

files = os.listdir(data_path)

# make a dict, key - file_name_last part
dta = dict()

for file in files:
    df = pd.read_csv(os.path.join(data_path, file), skiprows=config.COUNT_SKIP, sep=';')
    df = loader.fill_empty(loader.transform_header(df))
    splitted = loader.split(df.columns[1:])
    group = loader.group(splitted, df)

    dta[file] = group

In [ ]:
df = pd.read_csv(os.path.join(data_path, 'month_csv_acceleration_10khz.csv'), skiprows=config.COUNT_SKIP, sep=';')
df = loader.fill_empty(loader.transform_header(df))

In [ ]:
def tkeo_operator(data, k = 1):
    """ 
    Teager-Kaiser Energy operator
    """
    npnts = len(data[0])
    nsignals = len(data)
    filt_data = deepcopy(data)
    for i in range(nsignals):
        for n in range(k, npnts-k):
            filt_data[i][n] = data[i][n]**2-data[i][n-1]*data[i][n+1]
    return filt_data

def normilize(signal: np.ndarray):
    """
    MinMaxScaler + Teager-Kaiser Operator + Scaler
    """
    # scalers = [MinMaxScaler, StandardScaler]
    scaler = MinMaxScaler(feature_range=(0, 1))
    signal = scaler.fit_transform(signal)
    print(f'norm1 max: {signal.max()}, min: {signal.min()}')
    signal = tkeo_operator(signal)
    print(f'tkeo max: {signal.max()}, min: {signal.min()}')
    signal = scaler.fit_transform(signal)
    print(f'norm2 max: {signal.max()}, min: {signal.min()}')
    return signal


In [ ]:
series = normilize(df['3.Вибропреобразователь ППНг 3_2 СКЗ Виброускорения 10 кГц'].to_numpy().reshape(-1, 1))[:2000]
series_plot = normilize(df['3.Вибропреобразователь ППНг 3_2 СКЗ Виброускорения 10 кГц'].to_numpy().reshape(-1, 1))[2000:]

l, u = norm.interval(0.95, loc=series.mean(), scale=series.std())

plt.axhline(series.mean(), c='r', label='expected_value')
plt.fill_between(np.arange(10000), (series.mean() - l), (series.mean() + u), color='blue', alpha=.1)
plt.plot(np.arange(len(series_plot)), series_plot, label='signal')

In [ ]:
# df_ex = pd.read_csv(os.path.join(data_path, 'month_csv_excess.csv'), skiprows=config.COUNT_SKIP, sep=';')
# df_ex = loader.fill_empty(loader.transform_header(df_ex))

Этот график ничего не говорит о совместном распределении

In [ ]:
# # Генерация двух выборок (X и Y)
# np.random.seed(0)
# sample_size = 100
# X = df['1.Вибропреобразователь ПЗДг 5_2 СКЗ Виброускорения 10 кГц'].to_numpy()[:9100]
# Y = df_ex['1.Вибропреобразователь ПЗДг 5_2 Эксцесс'].to_numpy()[:9100]

# # Построение гистограммы двумерных данных с нормированными частотами
# plt.hist2d(X, Y, bins=100, cmap='Blues')
# plt.xlabel('X (Ускорение)')
# plt.ylabel('Y (Эксцесс)')
# plt.title('Совместное распределение вероятностей двумерной случайной величины (Ускорение, Эксцесс)')
# plt.colorbar(label='Частота')
# plt.show()

Тут скорее всего ошибка, потому что границы имеют странные значения

In [ ]:
# # Генерация исходных данных (ускорение и эксцесс)
# np.random.seed(0)
# size = 100
# acceleration = df['1.Вибропреобразователь ПЗДг 5_2 СКЗ Виброускорения 10 кГц'][:8000].to_numpy()
# excess = df_ex['1.Вибропреобразователь ПЗДг 5_2 Эксцесс'][:8000].to_numpy()

# # Функция для оценки доверительной области методом бутстрэпа
# def bootstrap_confidence_interval(data_x, data_y, n_bootstrap_samples=1000, alpha=0.05):
#     bootstrap_indices = np.random.choice(len(data_x), size=(n_bootstrap_samples, len(data_x)), replace=True)
#     bootstrap_means_x = np.mean(data_x[bootstrap_indices], axis=1)
#     bootstrap_means_y = np.mean(data_y[bootstrap_indices], axis=1)
#     sorted_means_x = np.sort(bootstrap_means_x)
#     sorted_means_y = np.sort(bootstrap_means_y)
#     lower_index = int((n_bootstrap_samples - 1) * alpha / 2)
#     upper_index = int((n_bootstrap_samples - 1) * (1 - alpha / 2))
#     lower_bound_x, upper_bound_x = sorted_means_x[lower_index], sorted_means_x[upper_index]
#     lower_bound_y, upper_bound_y = sorted_means_y[lower_index], sorted_means_y[upper_index]
#     return (lower_bound_x, upper_bound_x), (lower_bound_y, upper_bound_y)

# # Построение доверительной области
# lower_bounds, upper_bounds = bootstrap_confidence_interval(acceleration, excess)
# plt.scatter(acceleration, excess, label='Исходные данные')
# plt.xlabel('Ускорение')
# plt.ylabel('Эксцесс')
# plt.title('Доверительная область для двумерной случайной величины Эксцесс/Ускорение')
# plt.axhline(lower_bounds[1], color='r', linestyle='--', label='Нижняя граница (Эксцесс)')
# plt.axhline(upper_bounds[1], color='r', linestyle='--', label='Верхняя граница (Эксцесс)')
# plt.axvline(lower_bounds[0], color='g', linestyle='--', label='Нижняя граница (Ускорение)')
# plt.axvline(upper_bounds[0], color='g', linestyle='--', label='Верхняя граница (Ускорение)')
# plt.fill_between(np.arange(35), lower_bounds[1], upper_bounds[1], alpha=.1)
# plt.fill_between(np.arange(25), lower_bounds[0], upper_bounds[0], alpha=.1)
# # plt.legend()
# plt.grid(True)
# plt.show()


Генерируем полиномиальные признаки (Пока что их применимость остаётся под вопросов).  
Не интересуют сигналы (признаки), которые представляют собой x^2, x^3, ..., x^n, т.е. простое возведение в степень изначального сигнала

Полиномиальные признаки должны генерироваться по группам, потому что иначе смешиваются данные с разных датчиков

Признаки мусор, можно было не тратить время

In [ ]:
# fig, ax = plt.subplots(1, 1, figsize=(12, 12))

generator = PolynomialFeatures(degree=3, interaction_only=True)

# for item in generator.fit_transform(df.drop('date', axis=1)):
#     sns.lineplot(item)

# test = generator.fit_transform(df.drop('date', axis=1))
# print(generator.get_feature_names_out())
# for i, item in enumerate(test[:, 16:].T):
#     ax.plot(normilize(item.reshape(-1, 1)), label=i)

# plt.legend()

Рассмотрение параметров 10 кГц выборки

In [ ]:
def fit_poly(x, *args):
    params = args
    print(params)